# Explain Model Predictions with Amazon SageMaker Clarify

There are expanding business needs and legislative regulations that require explainations of _why_ a model mades the decision it did. SageMaker Clarify uses SHAP to explain the contribution that each input feature makes to the final decision.

In [ ]:
import boto3
import sagemaker
import pandas as pd
import numpy as np

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

# Test data for explainability

We created test data in JSONLines format to match the model inputs. 

In [ ]:
test_data_explainability_path = "./data-clarify/test_data_explainability.jsonl"

In [ ]:
!head -n 1 $test_data_explainability_path

### Upload the data

In [ ]:
test_data_explainablity_s3_uri = sess.upload_data(
    bucket=bucket, key_prefix="bias/test_data_explainability", path=test_data_explainability_path
)
test_data_explainablity_s3_uri

In [ ]:
!aws s3 ls $test_data_explainablity_s3_uri

In [ ]:
%store test_data_explainablity_s3_uri

# List Pipeline Execution Steps


In [ ]:
%store -r pipeline_name

In [ ]:
print(pipeline_name)

In [ ]:
%%time

import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

In [ ]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

In [ ]:
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]
print(pipeline_execution_arn)

In [ ]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

# View Created Model

In [ ]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "CreateModel":
        model_arn = execution_step["Metadata"]["Model"]["Arn"]
        break
print(model_arn)

pipeline_model_name = model_arn.split("/")[-1]
print(pipeline_model_name)

# Setup Model Explainability Analysis

In [ ]:
from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, instance_count=1, instance_type="ml.c5.2xlarge", sagemaker_session=sess
)

# Writing DataConfig and ModelConfig
A `DataConfig` object communicates some basic information about data I/O to Clarify. We specify where to find the input dataset, where to store the output, the target column (`label`), the header names, and the dataset type.

Similarly, the `ModelConfig` object communicates information about your trained model and `ModelPredictedLabelConfig` provides information on the format of your predictions.  

**Note**: To avoid additional traffic to your production models, SageMaker Clarify sets up and tears down a dedicated endpoint when processing. `ModelConfig` specifies your preferred instance type and instance count used to run your model on during Clarify's processing.

## DataConfig

In [ ]:
explainability_report_prefix = "bias/explainability-report-{}".format(pipeline_model_name)

explainability_output_path = "s3://{}/{}".format(bucket, explainability_report_prefix)

explainability_data_config = clarify.DataConfig(
    s3_data_input_path=test_data_explainablity_s3_uri,
    s3_output_path=explainability_output_path,
    headers=["review_body", "product_category"],
    features="features",
    dataset_type="application/jsonlines",
)

## ModelConfig

In [ ]:
model_config = clarify.ModelConfig(
    model_name=pipeline_model_name,
    instance_type="ml.m5.4xlarge",
    instance_count=1,
    content_type="application/jsonlines",
    accept_type="application/jsonlines",
    content_template='{"features":$features}',
)

## SHAPConfig

In [ ]:
shap_config = clarify.SHAPConfig(
    baseline=[{"features": ["ok", "Digital_Software"]}],  # [data.iloc[0].values.tolist()],
    num_samples=5,
    agg_method="mean_abs",
)

# Run Clarify Job

In [ ]:
clarify_processor.run_explainability(
    model_config=model_config,
    model_scores="predicted_label",
    data_config=explainability_data_config,
    explainability_config=shap_config,
    wait=False,
    logs=False,
)

In [ ]:
run_explainability_job_name = clarify_processor.latest_job.job_name
run_explainability_job_name

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, run_explainability_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, run_explainability_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}?prefix={}/">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, explainability_report_prefix
        )
    )
)

In [ ]:
from pprint import pprint

running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=run_explainability_job_name, sagemaker_session=sess
)

processing_job_description = running_processor.describe()

pprint(processing_job_description)

In [ ]:
running_processor.wait(logs=False)

# Download Report From S3

In [ ]:
!aws s3 ls $explainability_output_path/

In [ ]:
!aws s3 cp --recursive $explainability_output_path ./explainability_report/

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="./explainability_report/report.html">Explainability Report</a></b>'))

# View the Explainability Report
As with the bias report, you can view the explainability report in Studio under the experiments tab

<img src="img/explainability_detail.gif">

The Model Insights tab contains direct links to the report and model insights.

If you're not a Studio user yet, as with the Bias Report, you can access this report at the following S3 bucket.

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>